In [1]:
import numpy as np
import cv2

<h1 style = "text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d8/dfe/classcv_1_1VideoCapture.html">VideoCapture</a>
</h1>
<h2 style = "text-align: center">
is class for video capturing from video files, image sequences or cameras.
</h2>

In [2]:
vid = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')

sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

template_img = cv2.imread('./images/final_exam/Templates/Template-1.png')
template_gray = cv2.cvtColor(template_img,cv2.COLOR_BGR2GRAY)

min_match_number = 13

while vid.isOpened() :
    ret, frame = vid.read()

    query_img = frame
    query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)
    
    if ret :

        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        good_matches_list = list()
        for m, n in matches :
            if m.distance < 0.65*n.distance :     #เปรียบเทียบหาจำนวนgood matches
                good_matches.append(m)
                good_matches_list.append([m])
        
        if len(good_matches) > min_match_number :    #เปรียบเทียบจำนวนgood matches กับ min match ที่เรากำหนด ถ้ามันน้อยเกินจะจับภาพมั่ว
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)      
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2.0) # 2.0 คือค่าที่ยอมรับได้+-2 px
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
            drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

            cv2.imshow('red18+', detected_img)

        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()